# Mode MODIS Land Class
We'll use the last `n` years of MODIS land data to find the mode land class per pixel. This follows the document, which says that the uncertainty per year per pixel is too high to trust any individual year overly much.

```
Despite improving the stability to the product, we urge users not to use the product to determine post-classification land cover change. The amount of uncertainty in the land cover labels for any one year remains too high to distinguish real change from changes between classes that are spectrally indistinguishable at the coarse 500-m MODIS resolution. For more detailed information about the development and accuracy of the C6 MCD12Q1 product see Sulla-Menashe et al. (2018).
```

Source (accessed 2023-11-30): https://lpdaac.usgs.gov/documents/1409/MCD12_User_Guide_V61.pdf

In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
import _functions as pmf

In [2]:
# Specify where the config file can be found
config_file = '../0_config/config.txt'

In [4]:
# Get the required info from the config file
raw_path = pmf.read_from_config(config_file,'raw_path')
last_n_years = pmf.read_from_config(config_file,'last_n_years')

### 1. Find files

In [5]:
import glob

In [7]:
src_path = Path(raw_path) / 'modis_land' / 'raw'

In [20]:
modis_files = sorted( glob.glob(str(src_path / '*.tif')) )

### 2. Create the output directory

In [12]:
des_path = Path(raw_path) / 'modis_land' / 'annual_mode_2013_2022'

In [13]:
des_path.mkdir(exist_ok=True, parents=True)

### 3. Calculate mode land class

In [60]:
import numpy as np
import numpy.ma as ma
import os
from osgeo import gdal, osr
import scipy.stats as sc

In [24]:
gdal.UseExceptions()

#### 3.1 Functions

In [25]:
def get_geotif_data_as_array(file, band=1):
    ds = gdal.Open(file) # open the file
    band = ds.GetRasterBand(band) # get the data band
    data = band.ReadAsArray() # convert to numpy array for further manipulation   
    return data

In [26]:
def get_geotif_noData(src_file, band=1):
    src_ds = gdal.Open(src_file)
    src_band = src_ds.GetRasterBand(band)
    no_data_value = src_band.GetNoDataValue()
    src_ds = None
    return no_data_value

In [44]:
def write_geotif_sameDomain(src_file, des_file, des_data, no_data_value=None):
    
    # load the source file to get the appropriate attributes
    src_ds = gdal.Open(src_file)
    
    # get the geotransform
    des_transform = src_ds.GetGeoTransform()

    # Get the scale factor from the source metadata
    scale_factor = src_ds.GetRasterBand(1).GetScale()
    offset = src_ds.GetRasterBand(1).GetOffset()
    
    # get the data dimensions
    ncols = des_data.shape[1]
    nrows = des_data.shape[0]
    
    # make the file
    driver = gdal.GetDriverByName("GTiff")
    dst_ds = driver.Create(des_file,ncols,nrows,1,gdal.GDT_Float32, options = [ 'COMPRESS=DEFLATE' ])
    
    # Write the data
    #dst_ds.GetRasterBand(1).WriteArray( des_data )
    dst_band = dst_ds.GetRasterBand(1)
    dst_band.WriteArray(des_data)
    if no_data_value:
        dst_band.SetNoDataValue(no_data_value)
    
    # Set the scale factor and offset in the destination band, if they were defined in the source
    if scale_factor: dst_ds.GetRasterBand(1).SetScale(scale_factor)
    if offset: dst_ds.GetRasterBand(1).SetOffset(offset)
    
    # Set the geotransform
    dst_ds.SetGeoTransform(des_transform)

    # Set the projection
    wkt = src_ds.GetProjection()
    srs = osr.SpatialReference()
    srs.ImportFromWkt(wkt)
    dst_ds.SetProjection( srs.ExportToWkt() )
    
    # close files
    src_ds = None
    des_ds = None

    return

#### 3.2 Processing

In [21]:
# Subset files to last n years
modis_files = modis_files[-last_n_years:] 

In [29]:
# Load the data as numpy arrays, stack vertically, and find the mean value (ignoring nan)
data = [get_geotif_data_as_array(file) for file in modis_files] # Get data as uint8

In [31]:
stacked = np.dstack(data) # Create a 3D stack

In [39]:
mode_land_class = sc.mode(stacked, axis=2)

In [ ]:
# Get the noData value 
modis_noData = get_geotif_noData(modis_files[0])

In [42]:
# Transfer into masked array for GeoTIFF writing
modis_to_file = ma.masked_equal(mode_land_class[0], modis_noData)

In [58]:
# Save
first_data_year = os.path.basename(modis_files[0])[0:4] 
last_data_year = os.path.basename(modis_files[-1])[0:4]
des_name = f'{first_data_year}_{last_data_year}_mode{os.path.basename(modis_files[0])[8:]}'
des_file = str( des_path / des_name )

In [61]:
write_geotif_sameDomain(modis_files[0], des_file, modis_to_file.filled(), no_data_value=modis_noData)